In [1]:
from sqlalchemy import create_engine, text
from tabulate import tabulate

In [2]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [3]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [4]:
engine = create_engine(DATABASE_URL)

In [5]:
def execute_sql(sql_statement):
    try:
        with engine.connect() as connection:
            connection.execute(text(sql_statement))
            connection.commit()
            print(f"SQL statement executed successfully:\n{sql_statement}")
    except Exception as e:
        print(f"Error executing SQL statement:\n{sql_statement}\nError: {e}")

In [6]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Images (
        image_id SERIAL PRIMARY KEY,
        image_name VARCHAR(255),
        image_data BYTEA  -- For storing the image data
    );
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Images (
        image_id SERIAL PRIMARY KEY,
        image_name VARCHAR(255),
        image_data BYTEA  -- For storing the image data
    );



### Read Image Data from a File

In [ ]:
def read_image_data(image_path):
    """Reads binary image data from a file."""
    try:
        with open(image_path, 'rb') as image_file:
            image_data = image_file.read()
        return image_data
    except FileNotFoundError:
        print(f"Error: File not found at path: {image_path}")
        return None
    except Exception as e:
        print(f"Error reading image: {e}")
        return None

### Insert the Image Data into the Database

In [8]:
def insert_image(image_path, image_name):
    image_data = read_image_data(image_path)

    if image_data:
        try:
            with engine.connect() as connection:
                connection.execute(
                    text("""
                        INSERT INTO Images (image_name, image_data)
                        VALUES (:image_name, :image_data)
                    """),
                    {"image_name": image_name, "image_data": image_data}
                )
                connection.commit()
                print(f"Image '{image_name}' inserted successfully.")
        except Exception as e:
            print(f"Error inserting image: {e}")

In [9]:
image_path = "test.jpg"
image_name = "test.jpg"
insert_image(image_path, image_name)

Image 'test.jpg' inserted successfully.


### Retrieving the Image (Example)

In [ ]:
def retrieve_image(image_id):
    try:
        with engine.connect() as connection:
            result = connection.execute(
                text("""
                    SELECT image_data, image_name
                    FROM Images
                    WHERE image_id = :image_id
                """),
                {"image_id": image_id}
            )
            row = result.fetchone()

            if row:
                image_data = row[0]
                image_name = row[1]

                with open(f"retrieved_{image_name}", 'wb') as image_file:
                    image_file.write(image_data)
                print(f"Image with ID {image_id} retrieved and saved to retrieved_{image_name}")
            else:
                print(f"Image with ID {image_id} not found.")

    except Exception as e:
        print(f"Error retrieving image: {e}")

In [11]:
retrieve_image(1)

Image with ID 1 retrieved and saved to retrieved_test.jpg
